In [3]:
# Use env cxcr4
# Basically plot copykat results where we can compare with epithelial signature
# as well as SingleR predictions
# None of these plots get turned into figures but they're useful for reference
library(Seurat)
library(tidyverse)

In [ ]:
# index <- 18

for (index in seq(18, 40)) {
    if (index == 25) next();

    sample <- paste0('GM', index)
    message(paste('Starting sample', sample))


    samplepath <- paste0('/home/ubuntu/data/cxcr4-pdac/seurat/',
                        sample,'/', sample, '_cb.rds')
    fga_path <- paste0('/home/ubuntu/data/cxcr4-pdac/copykat/', 
                        sample, '/', sample, '_fga.tsv')
    copykat_pred_path <- paste0('/home/ubuntu/data/cxcr4-pdac/copykat/', 
                                sample, '/', sample, '_copykat_prediction.txt')
    outfile <- paste0('/home/ubuntu/data/cxcr4-pdac/copykat/', 
                                sample, '/', sample, '_copykat_correlation.pdf')


    obj <- readRDS(samplepath)
    fga <- read.table(fga_path, sep='\t', header=T) %>% column_to_rownames(var = 'barcode')
    copykat <- read.table(copykat_pred_path, sep='\t', header=T) %>% column_to_rownames(var = 'cell.names')
    obj <- obj %>% AddMetaData(fga) %>% AddMetaData(copykat)

    # ModuleScores
    # Panglao markers from Mori
    PDAC_epi <- list(c('AGR2','CTSE','GABRP','SFN','KLK10','PSCA','TSTD1','PIGR',
                    'LGALS4','FERMT1','RNF128','ECT2','CCL20','SPTSSB','VSIG2',
                    'AQP3','CDH3','KRT7','KRT16','KRT15','MUC1','MUC5AC','EPCAM',
                    'CEACAM1','ITGB4','ITGA2','TSPAN1','TSPAN8'))
    obj <- AddModuleScore(obj, features=PDAC_epi, name='PDAC_epi')

    tryCatch({
            while(T) {
                dev.off()
            }
        }, 
        error=function(e){}, 
        warning=function(w){}
    )

    message(paste('Outfile', outfile))
    pdf(outfile)
    
    r <- cor(obj$fga, obj$PDAC_epi1, use="complete.obs")
    (FeatureScatter(obj, 'fga', 'PDAC_epi1', group.by='copykat.pred') +
        ggtitle(paste0(sample, ': R = ', round(r, 3), '; R^2 = ', round(r**2, 3)))) %>% print()
    (DimPlot(obj, label=T) + 
        ggtitle(paste0(sample, ' Clusters'))) %>% print()
    (DimPlot(obj, label=T, group.by='celltype_hpca_main', repel=T) + 
        theme(legend.position = "none") + ggtitle(paste0(sample, ' HPCA Main Predictions'))) %>% print()
    (DimPlot(obj, group.by='copykat.pred') + 
        ggtitle(paste0(sample, ' Copykat predictions'))) %>% print()
    (FeaturePlot(obj, 'fga', min.cutoff=1e-500) + 
        ggtitle(paste0(sample, ' FGA'))) %>% print()
    (FeaturePlot(obj, 'PDAC_epi1', min.cutoff=1e-500) + 
        ggtitle(paste0(sample, ' PDAC_epi markers'))) %>% print()
    (VlnPlot(obj, 'fga', group.by='copykat.pred') + 
        ggtitle(paste0(sample, ' FGA by Copykat prediction'))) %>% print()
    (VlnPlot(obj, 'PDAC_epi1', group.by='copykat.pred') + 
        ggtitle(paste0(sample, ' PDAC_epi score by Copykat prediction'))) %>% print()
    dev.off()
}